In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_matrix_multiplication")' SchwiftyNotebook_matrix_multiplication

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_matrix_multiplication")
		SchwiftyNotebook_matrix_multiplication
With SwiftPM flags: []
Working in: /tmp/tmpm10836gp/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/h

In [5]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_matrix_multiplication

In [6]:
let thingo = Tensor([-2, -1, 0, 1, 2])

In [7]:
max(thingo, 0)

[0.0, 0.0, 0.0, 1.0, 2.0]


In [8]:
//export
public func reLU<T>(tensor: Tensor<T>) -> Tensor<T> where T : FloatingPoint, T: TensorFlowScalar {
    return max(tensor, 0)
}

In [9]:
//export
public func linearCombination<T>(inputs: Tensor<T>, weights: Tensor<T>, bias: Tensor<T>) -> Tensor<T> where T: FloatingPoint, T: TensorFlowScalar {
    return matmul(inputs, weights) + bias
}

In [10]:
let (xTrainingData, yTrainingData, xValidationData, yValidationData) = loadMNISTData(path: mnistPath)

In [33]:
let xTrainingDataNormalized = normalizeTensor(tensor: xTrainingData)
let xValidationDataNormalized = normalizeTensor(tensor: xValidationData)

In [62]:
assert(xTrainingDataNormalized.mean() < 0.001, "Expected to be less than zero")
assert(xValidationDataNormalized.mean() < 0.001, "Expected to be less than zero")
assert(1 - xTrainingDataNormalized.standardDeviation() < 0.001, "Expected to be less than zero")
assert(1 - xValidationDataNormalized.standardDeviation() < 0.001, "Expected to be less than zero")